# RAG Pipeline A/B Evaluation (Model-only)

This notebook loads your project and evaluates the **LLM-only** behavior of `rag.pipeline.RAGPipeline.answer` for a fixed set of queries, comparing two Ollama models (A/B).

In [1]:
import os

#OpenBLAS Warning : Detect OpenMP Loop and this application may hang...
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("GOTO_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

# silence HF advisory warnings & progress bars
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()

# (optional) quiet sentence-transformers too
import logging
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)


/home/sinedo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# RAG ENV VARIABLE

In [2]:
import os
def init_env_defaults() -> None:
    s = os.environ.setdefault
    nproc = str(os.cpu_count() or 8)
    # ---- defaults (override via env) ----
    
    s("RAG_ART_INDEX_PATH", "src/st3405_data/index/st4305_text_bgem3.faiss")
    s("RAG_ART_STORE_PATH", "src/st3405_data/index/st4305_store.pkl.gz")
    s("RAG_CTX_BUDGET_CHARS", "10000")
    
    s("RAG_HYBRID_ALPHA", "0.5")      # weight for dense in fusion (0..1)
    s("RAG_QUERY_EXPAND", "1")        # 1=enable LLM query reformulation via Ollama
    s("RAG_QUERY_EXPAND_K", "0.5")    # keep as in script
    s("RAG_PRF_ENABLE", "1")
    s("RAG_PRF_TERMS", "8")
    #
    # s("RAG_OLLAMA_MODEL", "llama3:latest")
    s("RAG_OLLAMA_MODEL", "llama3.2:3b")
    s("RAG_RERANK_TIMEOUT", "15.0")
    s("RAG_SELECT_TIMEOUT", "10.0")
    s("RAG_QUERY_VARIATIONS", "10")

    s("RAG_OLLAMA_NUM_PREDICT", "512")

    s("RAG_MIN_CTX", "2")
    s("RAG_MAX_CTX", "3")

    s("RAG_CANDIDATE_K_FOR_RERANK", "10")
    s("RAG_FINAL_TOP_N", "3")

    s("RAG_DISABLE_SELECT", "1")

    # Models / batching
    
    s("RAG_EMBEDDER_MODEL", "BAAI/bge-m3")
    s("RAG_EMBED_DEVICE", "cpu")
    s("RAG_EMBED_BATCH", "1")    
    s("RAG_EMBED_MAX_LENGTH", "256")

    #"BAAI/bge-reranker-base"
    # or jinaai/jina-reranker-v2-base-multilingual
    #naver/xprovence-reranker-bgem3-v1
    #naver/provence-reranker-debertav3-v1
    s("RAG_RERANKER_MODEL", "BAAI/bge-reranker-base")  
    
    # s("RAG_RERANKER_DEVICE", "cuda")
    # s("RAG_RERANKER_WINDOW", "384")
    # s("RAG_RERANKER_STRIDE", "256")
    # s("RAG_RERANKER_FP16", "1")
    # s("RAG_RERANKER_PAD_MAX", "0")
    # s("RAG_RERANKER_WINDOW_BATCH", "1")

    #cpu
    s("RAG_RERANKER_DEVICE", "cpu")
    s("RAG_RERANKER_WINDOW", "384")
    s("RAG_RERANKER_STRIDE", "256")
    s("RAG_RERANKER_FP16", "0")
    s("RAG_RERANKER_WINDOW_BATCH", "8")    
    s("RAG_RERANKER_QUANTIZE", "1")      
    s("TOKENIZERS_PARALLELISM", "true")    
    s("OMP_NUM_THREADS", nproc)
    s("MKL_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))
    s("OPENBLAS_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))
    s("NUMEXPR_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))

    s("RAG_RERANKER_MAX_LEN", "384")
    s("RAG_RERANKER_AGG", "max")    

    s("RAG_DISABLE_RERANK", "1")
    s("RAG_MAX_VARIANTS", "4")

    # Timeouts / perf
    s("RAG_EXPAND_TIMEOUT", "10.0")
    s("RAG_BM25_PARALLEL", "1")
    s("RAG_BM25_WORKERS", "16")

    s("RAG_DENSE_K_PER_QUERY", "10")
    s("RAG_SPARSE_K_PER_QUERY", "10")
    s("RAG_ENV_DUMPED", "0")
    s("RAG_FAISS_GPU", "0")

    # CUDA allocator
    s("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
    s("CUDA_VISIBLE_DEVICES", "1")
    
    s("KMP_AFFINITY", "granularity=fine,compact,1,0")

    try:
        import torch
        torch.set_grad_enabled(False)
        torch.set_num_threads(int(os.environ["OMP_NUM_THREADS"]))
        torch.set_num_interop_threads(1)
    except Exception:
        pass



# Chame no startup da sua app:
init_env_defaults()


In [3]:
import os
from pathlib import Path
import sys
from pathlib import Path

def get_root_path():
    """Always use the same, absolute (relative to root) paths

    which makes moving the notebooks around easier.
    """
    return Path(os.getcwd()).parent

PROJECT_DIR = Path(get_root_path())
assert PROJECT_DIR.exists(), PROJECT_DIR

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

print("Using project at:", PROJECT_DIR)

Using project at: /mnt/dados/projetos/gps-tracking-rag-assistent-v1/src


# Avaliação do RagPipeline com Conjunto de Perguntas (Homologação)

In [4]:

import json
from pathlib import Path
import pandas as pd

DATASET = Path("eval_queries.jsonl")
RESULTS_DIR = Path("eval_results"); RESULTS_DIR.mkdir(exist_ok=True, parents=True)

rows = [json.loads(line) for line in DATASET.read_text(encoding="utf-8").splitlines()]
df = pd.DataFrame(rows)

display(df.head(10))
print("Total de exemplos:", len(df))

,id,category,query,answer,gold_doc_ids,gold_refs_hint,section_key
0,Q01,Comandos/Parâmetros,Qual o comando para reiniciar o dispositivo vi...,Use o comando 'Reboot' (reinicia o dispositivo).,[20.envio de comandos],[20. ENVIO DE COMANDOS — Comando Reboot.],20.envio de comandos
1,Q02,Comandos/Parâmetros,Qual o comando para habilitar a Saída 1?,Use o comando 'Enable1' (ativa a Saída 1).,[20.envio de comandos],[20. ENVIO DE COMANDOS — Enable1 / Disable1.],20.envio de comandos
2,Q03,Comandos/Parâmetros,Como consultar a versão do firmware do rastrea...,Envie 'ReqVer' com Option=1 para solicitar a v...,[20.envio de comandos],[20. ENVIO DE COMANDOS — ReqVer (Option=1).],20.envio de comandos
3,Q04,Comandos/Parâmetros,Como iniciar a calibração DPA por comando?,Use 'Start DPA Calibration' para iniciar e 'St...,[20.envio de comandos],[20. ENVIO DE COMANDOS — Comandos de calibraçã...,20.envio de comandos
4,Q05,Comandos/Parâmetros,Existe comando para consultar o status do anti...,Sim. Utilize 'Get anti theft status' para cons...,[20.envio de comandos],[20. ENVIO DE COMANDOS — Consulta antifurto.],20.envio de comandos
5,Q06,Configurações de interface,Como habilitar o Fine Tracking no ST4305?,Parâmetro (3010) — Habilitar Fine Tracking: 01...,[27.configurando fine tracking],[27. CONFIGURANDO FINE TRACKING — (3010) Habil...,27.configurando fine tracking
6,Q07,Configurações de interface,Qual parâmetro define o intervalo entre posiçõ...,Parâmetro (3011) — Intervalo de posições GPS e...,[27.configurando fine tracking],[27. CONFIGURANDO FINE TRACKING — (3011) Inter...,27.configurando fine tracking
7,Q08,Configurações de interface,Como definir a quantidade de posições enviadas...,Parâmetro (3012) — Quantidade de posições por ...,[27.configurando fine tracking],[27. CONFIGURANDO FINE TRACKING — (3012) Quant...,27.configurando fine tracking
8,Q09,Configurações de interface,Quais campos podem ser habilitados no cabeçalh...,"No modo Small Table, é possível habilitar camp...",[24.configuração de cabeçalhos (stt e alt)],[24. CONFIGURAÇÃO DE CABEÇALHOS (STT E ALT) — ...,24.configuração de cabeçalhos (stt e alt)
9,Q10,Configurações de interface,Como configurar a APN de dados (parâmetros de ...,"Na seção de Parâmetro de Rede, configure APN, ...",[7.parâmetro de rede],[7. PARÂMETRO DE REDE — APN/dados móveis.],7.parâmetro de rede


Total de exemplos: 20


# RagPipeline

In [5]:
# src/api/api_manager.py
from __future__ import annotations

import gzip
import os
import pickle
from pathlib import Path
from typing import List, Dict

import faiss
from rank_bm25 import BM25Okapi

# RAG plumbing (optimized service uses sentence selection + compact prompts)
from rag.config import RetrievalConfig
from rag.pipeline import RAGPipeline
from rag.config import RetrievalConfig, Timeouts, BuildLimits

# Rerankers / LLM / embedder
from rag.rerank.hf_cross_encoder import HFCrossEncoderLongReranker
from rag.ollama import OllamaClient
from rag.init import get_embedder

# Tokenizer used to build/search BM25
from rag.utils import tokenize

# Retrievers
from rag.retrievers.dense_faiss import FaissRetriever
from rag.retrievers.sparse_bm25 import BM25Retriever
from rag.retrievers.multiquery_hybrid import MultiQueryHybridRetriever
from rag.query_expander import QueryExpander
from rag.types import DocStore

@staticmethod
def _env_int(name: str, default: int) -> int:
    try:
        return int(os.getenv(name, default))
    except Exception:
        return default

In [6]:
# Load FAISS index (GPU if available) ----
faiss_path = PROJECT_DIR / "st3405_data/index/st4305_text_bgem3.faiss"
if not Path(faiss_path).exists():
    raise FileNotFoundError(f"FAISS index not found at {faiss_path}")

index = faiss.read_index(str(faiss_path))
try:
    if os.getenv("RAG_FAISS_GPU", "0") == "1":
        if faiss.get_num_gpus() > 0:
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
            index_device = "gpu"
except Exception:
    # GPU not available or FAISS GPU build not installed → stay on CPU
    index_device = "cpu"

# IVF/HNSW search breadth (no-op for Flat)
try:
    if hasattr(index, "nprobe"):
        index.nprobe = max(32, int(0.1 * getattr(index, "nlist", 100)))
    if hasattr(index, "hnsw"):
        index.hnsw.efSearch = int(os.getenv("RAG_FAISS_EFSEARCH", "256"))
except Exception:
    pass

# Load store: docs (text), meta, ids, and stored model name ----
store_path = PROJECT_DIR / "st3405_data/index/st4305_store.pkl.gz"
if not Path(store_path).exists():
    raise FileNotFoundError(f"Store file not found at {store_path}")

with gzip.open(store_path, "rb") as f:
    store = pickle.load(f)

docs: List[str] = store["docs"]
meta: List[dict] = store["meta"]
ids:  List[str] = store["ids"]

embedder_model_name = store.get("model", os.getenv("RAG_EMBEDDER_MODEL"))

assert index.ntotal == len(docs) == len(meta) == len(ids), (
    f"Index/store size mismatch: faiss={index.ntotal}, docs={len(docs)}, meta={len(meta)}, ids={len(ids)}"
)

# id -> metadata/text
id_to_meta = {ids[i]: {**meta[i], "doc_id": ids[i]} for i in range(len(ids))}
doc_store: DocStore = {ids[i]: docs[i] for i in range(len(ids))}

# ---- 4) Embedder (normalize=True for cosine/IP) ----
emb_model = get_embedder(embedder_model_name, use_fp16=True, normalize=True)

# ---- 5) BM25 over the docs using the SAME tokenizer ----
tokenized = [tokenize(t) for t in docs]
bm25 = BM25Okapi(
    tokenized,
    k1=float(os.getenv("RAG_BM25_K1", "1.4")),
    b=float(os.getenv("RAG_BM25_B", "0.4")),
)

# ---- 6) LLM client (Ollama) ----
ollama = OllamaClient(
    base_url=os.getenv("RAG_OLLAMA_BASE_URL", "http://localhost:11434"),
    model=os.getenv("RAG_OLLAMA_MODEL", "llama3:latest"),
    timeout=int(os.getenv("RAG_OLLAMA_TIMEOUT", "120")),
)

# ---- 7) Retrieval configuration ----
cfg = RetrievalConfig(
    query_variations=_env_int("RAG_QUERY_VARIATIONS", 5),
    dense_k_per_query=_env_int("RAG_DENSE_K_PER_QUERY", 80),
    sparse_k_per_query=_env_int("RAG_SPARSE_K_PER_QUERY", 80),
    rrf_k_const=_env_int("RAG_RRF_K_CONST", 50),
    candidate_k_for_rerank=_env_int("RAG_CANDIDATE_K_FOR_RERANK", 160),
    final_top_n=_env_int("RAG_FINAL_TOP_N", 12),
)

# ---- 8) Build retriever stack (dense + sparse → per-query hybrid → multi-query hybrid) ----
dense = FaissRetriever(faiss_index=index, id_map=ids, embedder=emb_model)
sparse = BM25Retriever(bm25=bm25, id_map=ids, tokenize_fn=tokenize)

use_expand = os.getenv("RAG_QUERY_EXPAND", "1") == "1"
expander = QueryExpander(llm=ollama) if use_expand else None

retriever = MultiQueryHybridRetriever(
    dense=dense,
    sparse=sparse,
    expander=expander,
    rrf_k=cfg.rrf_k_const,
    per_query_k=max(cfg.dense_k_per_query, cfg.sparse_k_per_query),
    final_limit=cfg.candidate_k_for_rerank,
)

# ---- 10) Build the optimized RAG service ----

tmo = Timeouts()          # uses env defaults
limits = BuildLimits()    # uses env defaults

rag_pipeline = RAGPipeline(
    retriever=retriever,
    embedder=emb_model,
    llm=ollama,
    doc_store=doc_store,
    cfg=cfg,
    tmo=tmo,
    limits=limits,
    meta_store=id_to_meta
)

# Métricas (Retrieval & Ranking)

In [7]:
import math

def precision_at_k(relevant: set, ranked_ids: list, k: int = 5):
    k = min(k, len(ranked_ids))
    if k == 0: return 0.0
    hits = sum(1 for i in range(k) if ranked_ids[i] in relevant)
    return hits / k

def recall_at_k(relevant: set, ranked_ids: list, k: int = 5):
    if not relevant: return 0.0
    k = min(k, len(ranked_ids))
    hits = sum(1 for i in range(k) if ranked_ids[i] in relevant)
    return hits / len(relevant)

def mrr(relevant: set, ranked_ids: list):
    for idx, rid in enumerate(ranked_ids, start=1):
        if rid in relevant:
            return 1.0 / idx
    return 0.0

def hit_rate_at_k(relevant: set, ranked_ids: list, k: int = 5):
    k = min(k, len(ranked_ids))
    return 1.0 if any(ranked_ids[i] in relevant for i in range(k)) else 0.0

def dcg_at_k(gains: list, k: int):
    return sum((gains[i] / math.log2(i+2)) for i in range(min(k, len(gains))))

def ndcg_at_k(relevant_ordered: list, ranked_ids: list, k: int = 5):
    ideal = [1.0]*len(relevant_ordered)  # binário
    gains = [1.0 if rid in set(relevant_ordered) else 0.0 for rid in ranked_ids]
    idcg = dcg_at_k(ideal, k)
    if idcg == 0: return 0.0
    return dcg_at_k(gains, k) / idcg

# Métricas (Geração de Texto) — BLEU e ROUGE-L

In [8]:
def ngrams(tokens, n): 
    return list(zip(*[tokens[i:] for i in range(n)]))

def bleu(candidate, reference, max_n=4):
    cand, ref = candidate.split(), reference.split()
    if not cand: return 0.0
    precisions = []
    for n in range(1, max_n+1):
        cand_ngrams, ref_ngrams = ngrams(cand, n), ngrams(ref, n)
        ref_counts = {ng:ref_ngrams.count(ng) for ng in ref_ngrams}
        match = sum(min(cand_ngrams.count(ng), ref_counts.get(ng,0)) for ng in set(cand_ngrams))
        precisions.append(match / max(1, len(cand_ngrams)))
    bp = 1.0 if len(cand)>len(ref) else math.exp(1 - len(ref)/max(1,len(cand)))
    gm = math.exp(sum(math.log(p) for p in precisions if p>0)/len(precisions)) if any(p>0 for p in precisions) else 0.0
    return bp*gm

def _lcs(a,b):
    la, lb = len(a), len(b)
    dp=[[0]*(lb+1) for _ in range(la+1)]
    for i in range(la):
        for j in range(lb):
            if a[i]==b[j]: dp[i+1][j+1]=dp[i][j]+1
            else: dp[i+1][j+1]=max(dp[i][j+1],dp[i+1][j])
    return dp[la][lb]

def rouge_l(candidate, reference):
    cand, ref = candidate.split(), reference.split()
    if not cand or not ref: return 0.0
    lcs = _lcs(cand, ref)
    prec, rec = lcs/len(cand), lcs/len(ref)
    beta = 1.2**2
    return ((1+beta)*prec*rec)/(rec+beta*prec) if (rec+beta*prec)>0 else 0.0

# Execução da Avaliação

# Normalização e construção do GOLD

In [9]:
import unicodedata
from typing import Dict, Iterable, List, Set
import re

def _norm_id(x: str) -> str:
    """Normaliza identificadores de documentos de forma agnóstica."""
    if not isinstance(x, str):
        x = str(x)
    return unicodedata.normalize("NFKC", x).strip().lower()

def normalize_ids(ids: Iterable[str]) -> List[str]:
    """Aplica normalização a uma lista de IDs."""
    return [_norm_id(i) for i in ids if i is not None]

def build_gold_map(row) -> Dict[str, float]:
    """
    Constrói o mapa GOLD de doc_id -> ganho (peso).
    - Usa APENAS section_key e gold_doc_ids do dataset.
    - Suporta opcionalmente 'gold_gains' (dict) para pesos graduados.
    """
    gold_ids: Set[str] = set()

    # 1) section_key (string) — um único doc “correto” principal
    sk = row.get("section_key")
    if isinstance(sk, str) and sk.strip():
        gold_ids.add(_norm_id(sk))

    # 2) gold_doc_ids (lista) — outros docs aceitos como corretos
    g = row.get("gold_doc_ids")
    if isinstance(g, list):
        for x in g:
            if isinstance(x, str) and x.strip():
                gold_ids.add(_norm_id(x))

    # 3) Se nada foi especificado, forçamos correção do dataset (falha explícita)
    if not gold_ids:
        raise ValueError(
            f"[Dataset inválido] A linha id={row.get('id')} não tem 'section_key' nem 'gold_doc_ids'. "
            "Preencha um dos dois para calcular métricas corretamente."
        )

    # 4) Ganhos graduados (opcional). Se não vier, tudo = 1.0 (binário).
    gains_cfg = row.get("gold_gains") or {}
    gold_map: Dict[str, float] = {}
    for doc_id in gold_ids:
        gain = gains_cfg.get(doc_id, 1.0)
        try:
            gain = float(gain)
        except Exception:
            gain = 1.0
        gold_map[doc_id] = gain

    return gold_map

def command_match_ratio(gold_answer: str, pred_answer: str) -> float:
    """
    Retorna proporção de comandos 'CMD;..' presentes no pred_answer.
    Se não houver comandos no gold, retorna NaN.
    """
    cmds = re.findall(r"[A-Z]{2,4};[0-9;]+", gold_answer or "")
    if not cmds:
        return float("nan")
    pred = pred_answer or ""
    hits = sum(1 for c in cmds if c in pred)
    return hits / len(cmds)

# Utilitarios para 



In [10]:
# ==== Detectores + Métricas adaptativas para geração ====
import re
import math
from typing import Dict, List, Tuple
import unicodedata

_CMD_RE = re.compile(r"[A-Z]{2,4};[0-9;]+")
_PARAM_ID_RE = re.compile(r"\((\d{3,4})\)")   # captura (3010), (1055), etc.
_NUM_UNIT_RE = re.compile(
    r"(?P<num>(?:\d+[.,]?\d*))\s*(?P<unit>mhz|dbm|v|vc{1,2}|s|ms|km/h|ma|µa|mah|ghz|kbps|mbps)\b",
    flags=re.IGNORECASE
)
_BAND_RE = re.compile(r"\bband(as)?\s*:?|\b\b(?:banda|band)\b", re.IGNORECASE)

def _norm_txt(x: str) -> str:
    if not isinstance(x, str): x = str(x or "")
    return unicodedata.normalize("NFKC", x).strip()

def detect_qtype(row) -> str:
    """Heurística agnóstica baseada no GOLD (answer + hint + query)."""
    gold = " ".join([
        _norm_txt(row.get("answer","")),
        " ".join(row.get("gold_refs_hint", [])) if isinstance(row.get("gold_refs_hint"), list) else _norm_txt(row.get("gold_refs_hint","")),
        _norm_txt(row.get("query",""))
    ])
    if _CMD_RE.search(gold) or re.search(r"\b(reboot|reqimsi|reqver|enable\d|disable\d|preset|initmsgno)\b", gold, re.I):
        return "command"
    if _PARAM_ID_RE.search(gold) or re.search(r"\b(apn|porta|cabeçalho|stt|table|perfil)\b", gold, re.I):
        return "config"
    if _NUM_UNIT_RE.search(gold) or _BAND_RE.search(gold) or re.search(r"\bmhz|dbm|vcc|v\b", gold, re.I):
        return "specs"
    # fallback sem penalizar
    return "general"

def extract_cmds(text: str) -> List[str]:
    return _CMD_RE.findall(text or "")

def command_match_ratio(gold_answer: str, pred_answer: str) -> float:
    g = extract_cmds(gold_answer); 
    if not g: return float("nan")
    hits = sum(1 for c in g if c in (pred_answer or ""))
    return hits/len(g)

def command_exact_match(gold_answer: str, pred_answer: str) -> float:
    g, p = set(extract_cmds(gold_answer)), set(extract_cmds(pred_answer))
    if not g: return float("nan")
    return 1.0 if p.issuperset(g) else 0.0

def command_jaccard(gold_answer: str, pred_answer: str) -> float:
    g, p = set(extract_cmds(gold_answer)), set(extract_cmds(pred_answer))
    if not g and not p: return float("nan")
    inter = len(g & p); uni = len(g | p)
    return (inter/uni) if uni else float("nan")

def extract_numbers_units(text: str) -> List[Tuple[str,str]]:
    """Retorna lista de (numero_normalizado, unidade_normalizada)."""
    res = []
    for m in _NUM_UNIT_RE.finditer(text or ""):
        num = m.group("num").replace(",", ".")
        unit = m.group("unit").lower()
        res.append((num, unit))
    return res

def numeric_unit_match(gold_answer: str, pred_answer: str) -> float:
    """Proporção de (número,unidade) do GOLD presentes no pred (como substring)."""
    gold_pairs = extract_numbers_units(gold_answer)
    if not gold_pairs: 
        return float("nan")
    pred = (pred_answer or "").lower()
    hits = 0
    for num, unit in gold_pairs:
        if (num.lower() in pred) and (unit.lower() in pred):
            hits += 1
    return hits/len(gold_pairs)

def list_overlap_ratio(gold_items: List[str], pred_text: str) -> float:
    """Checa a presença de cada item (ex.: bandas LTE) no texto predito (case-insensitive)."""
    if not gold_items:
        return float("nan")
    pred = (pred_text or "").lower()
    hits = sum(1 for it in gold_items if _norm_txt(it).lower() in pred)
    return hits/len(gold_items)

def bands_from_text(text: str) -> List[str]:
    """Extrai bandas do tipo '1 [2100MHz], 3 [1800MHz], 5 [850MHz], 28 [700MHz]' como lista '1','3','5','28'."""
    if not isinstance(text, str): return []
    bands = re.findall(r"\b(\d{1,2})\s*\[\s*\d+\s*mhz\s*\]", text, flags=re.I)
    if bands:
        return bands
    # fallback: só números isolados precedidos de 'banda'/'band'
    if _BAND_RE.search(text or ""):
        bands = re.findall(r"\b(\d{1,2})\b", text)
    return bands

def compute_generation_metrics(row, pred_answer: str) -> Dict[str, float]:
    """Retorna métricas adequadas ao tipo detectado. NaN quando não aplicável."""
    gold_answer = _norm_txt(row.get("answer",""))
    qtype = detect_qtype(row)
    out = {"gen_type": qtype}

    # if qtype == "command":
    #     out["cmd_match"]   = command_match_ratio(gold_answer, pred_answer)
    #     out["cmd_exact"]   = command_exact_match(gold_answer, pred_answer)
    #     out["cmd_jaccard"] = command_jaccard(gold_answer, pred_answer)
    #     # BLEU/ROUGE se quiser manter:
    #     out["bleu"]    = bleu(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     out["rouge_l"] = rouge_l(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     return out

    # if qtype == "config":
    #     out["num_unit_match"] = numeric_unit_match(gold_answer, pred_answer)
    #     # checagem de presença de IDs de parâmetros (3010 etc.)
    #     gold_pids = set(_PARAM_ID_RE.findall(gold_answer))
    #     if gold_pids:
    #         hits = sum(1 for pid in gold_pids if f"({pid})" in (pred_answer or ""))
    #         out["param_id_cover"] = hits/len(gold_pids)
    #     else:
    #         out["param_id_cover"] = float("nan")
    #     out["bleu"]    = bleu(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     out["rouge_l"] = rouge_l(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     return out

    # if qtype == "specs":        
    #     out["bleu"]    = bleu(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     out["rouge_l"] = rouge_l(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    #     return out

    # general
    out["bleu"]    = bleu(pred_answer, gold_answer) if gold_answer.strip() else float("nan")
    out["rouge_l"] = rouge_l(pred_answer, gold_answer) if gold_answer.strip() else float("nan")

    return out

def safe_bertscore(hyp: str, ref: str, lang: str = "pt"):
    from bert_score import score as bertscore_score
    """
    Retorna (P, R, F1) como floats ou (None, None, None) se indisponível.
    """
    hyp = hyp or ""
    ref = ref or ""
    try:
        P, R, F1 = bertscore_score(
            [hyp], [ref],
            lang=lang,
            rescale_with_baseline=True,
            batch_size=8,
            device="cpu",
        )
        # tensores -> float
        return float(P.mean()), float(R.mean()), float(F1.mean())
    except Exception as e:
        print(f"⚠️ Erro ao calcular BERTScore: {e}")
        return None, None, None



In [11]:
os.environ.setdefault("RAG_LOG_ENABLED","0")

'0'

# Avaliacao das metricas

In [12]:
# ==== Execução (N questões) + métricas (inclui BERTScore) + salvamento em JSONL ====
import time, json
from pathlib import Path
import pandas as pd


K_LIST = [1, 3, 5]
records, latencies = [], []

RESULTS_DIR = RESULTS_DIR
res_path = RESULTS_DIR / "rag_pipeline_results.jsonl"


# Limpa arquivo anterior (opcional)
if res_path.exists():
    res_path.unlink()

rows_iter = df.iterrows()  # ex.: df.iterrows() para todas

for i, row in rows_iter:
    qid         = row["id"]
    query       = row["query"]
    gold_answer = row.get("answer", "") or ""

    print(f"\n🔍 Pergunta {qid}: {query}")

    # Execução do pipeline (end-to-end)
    t0 = time.time()
    result = rag_pipeline.answer(query)
    t1 = time.time()
    lat = t1 - t0
    latencies.append(lat)

    # Resposta e docs retornados pelo pipeline
    answer          = (result.get("answer") or "").strip()
    ranked_ids_raw  = result.get("docs", []) or []
    ranked_ids      = normalize_ids(ranked_ids_raw)   # <<< sua normalização
    top1_score      = float(result.get("score_top1", 0.0) or 0.0)

    #print(f"\n🧠 Resposta gerada:\n{answer}\n")
    #print(f"📄 Docs recuperados: {ranked_ids}")

    # === GOLD determinístico (sem heurística/df_docs) ===
    gold_map = build_gold_map(row)     # dict: doc_id_normalizado -> ganho
    relevant = set(gold_map.keys())    # conjunto para métricas binárias

    # === Métricas de retrieval ===
    rec = {
        "id": qid,
        "category": row.get("category", ""),
        **{f"precision@{k}": precision_at_k(relevant, ranked_ids, k) for k in K_LIST},
        **{f"recall@{k}":    recall_at_k(relevant,  ranked_ids, k) for k in K_LIST},
        "mrr": mrr(relevant, ranked_ids),
        **{f"hit@{k}":  hit_rate_at_k(relevant, ranked_ids, k) for k in K_LIST},
        **{f"ndcg@{k}": ndcg_at_k(list(relevant), ranked_ids, k) for k in K_LIST},
    }

    # === Métricas de geração existentes (ex.: BLEU/ROUGE) ===
    genm = compute_generation_metrics(row, answer)  # mantém sua função atual
    rec.update(genm)

    # === BERTScore (P/R/F1) por questão ===
    bsP, bsR, bsF1 = safe_bertscore(answer, gold_answer, lang="pt")
    rec["bertscore_P"]  = bsP
    rec["bertscore_R"]  = bsR
    rec["bertscore_F1"] = bsF1

    # Extras úteis para auditoria
    rec["gen_answer"]     = answer
    rec["gold_answer"]    = gold_answer
    rec["score_top1"]     = top1_score
    rec["docs"]           = ranked_ids
    rec["latency_sec"]    = lat
    rec["gold_refs_hint"] = row["gold_refs_hint"] if "gold_refs_hint" in row else ""

    records.append(rec)

# Tabela consolidada
records_df = pd.DataFrame(records)

# Salvar JSONL
with res_path.open("w", encoding="utf-8") as f:
    for r in records:
        json.dump(r, f, ensure_ascii=False)
        f.write("\n")

# Sumário rápido
avg_latency = (sum(latencies) / len(latencies)) if latencies else float("nan")
print(f"\n✅ Resultados salvos em: {res_path}")
print(f"⏱️ Latência média (s/query): {avg_latency:.3f}")
print(f"🧪 Linhas avaliadas: {len(records_df)}")



🔍 Pergunta Q01: Qual o comando para reiniciar o dispositivo via envio de comandos?


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 11.86it/s]



🔍 Pergunta Q02: Qual o comando para habilitar a Saída 1?


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 10.72it/s]



🔍 Pergunta Q03: Como consultar a versão do firmware do rastreador por comando?


...


🔍 Pergunta Q19: Qual a capacidade de memória FIFO de posições?


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 10.73it/s]



🔍 Pergunta Q20: O produto é homologado por algum órgão regulatório?


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 12.66it/s]



✅ Resultados salvos em: eval_results/rag_pipeline_results.jsonl
⏱️ Latência média (s/query): 17.175
🧪 Linhas avaliadas: 20


# Sumários (Global e por Categoria) em .jsonl

In [13]:
# ==== Sumários Global e por Categoria em JSONL ====
import pandas as pd
import json

DEC = 3 

res_path = RESULTS_DIR / "rag_pipeline_results.jsonl"
sum_cat_path = RESULTS_DIR / "summary_by_category.jsonl"
sum_glb_path = RESULTS_DIR / "summary_global.jsonl"

# Carrega o JSONL recem-salvo
pred = [json.loads(l) for l in res_path.read_text(encoding="utf-8").splitlines() if l.strip()]
df_pred = pd.DataFrame(pred)

agg_cols = [c for c in [
    "precision@1","precision@3","precision@5",
    "recall@1","recall@3","recall@5",
    "mrr","hit@1","hit@3","hit@5",
    "ndcg@1","ndcg@3","ndcg@5",
    "bleu","rouge_l",
    "bertscore_P","bertscore_R","bertscore_F1",
] if c in df_pred.columns]

# Global
summary_global = df_pred[agg_cols].mean(numeric_only=True).to_frame("mean").reset_index().rename(columns={"index":"metric"})
summary_global["mean"] = summary_global["mean"].round(DEC)

# Por categoria (se houver)
cat_col = "category" if "category" in df_pred.columns else None
if cat_col:
    summary_by_cat = df_pred.groupby(cat_col)[agg_cols].mean(numeric_only=True).reset_index()
    summary_by_cat[agg_cols] = summary_by_cat[agg_cols].round(DEC)
else:
    summary_by_cat = pd.DataFrame()

# Salvar como JSONL
with sum_glb_path.open("w", encoding="utf-8") as f:
    for _, row in summary_global.iterrows():
        json.dump(row.to_dict(), f, ensure_ascii=False)
        f.write("\n")

with sum_cat_path.open("w", encoding="utf-8") as f:
    for _, row in summary_by_cat.iterrows():
        json.dump(row.to_dict(), f, ensure_ascii=False)
        f.write("\n")

print(f"✅ summary_global.jsonl → {sum_glb_path}")
print(f"✅ summary_by_category.jsonl → {sum_cat_path}")


✅ summary_global.jsonl → eval_results/summary_global.jsonl
✅ summary_by_category.jsonl → eval_results/summary_by_category.jsonl
